In [1]:
# Funciones caseras para estandarizar la computación
#from Functions import *
# Para el acceso a los ficheros
import os
# Para las imágenes
import numpy as np
import cv2 # OpenCV-Python
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
# Para controlar el tiempo
import time

In [2]:
# SIFT feature detector/descriptor
sift = cv2.xfeatures2d.SIFT_create()

In [3]:
data_path = "../Circuito_Almeria_171021/Iniciados/Data_resized/"
examples_path = "../Circuito_Almeria_171021/Iniciados/Examples/"

In [4]:
moto_example = cv2.imread(data_path + "1" + "/" + os.listdir(data_path + os.listdir(data_path)[0])[5])

In [22]:
sift_points_examples_list = list()
folder_names_list = list()
for example_name in os.listdir(examples_path):
    folder_names_list.append(example_name[:-4])
    example = cv2.cvtColor(cv2.imread(os.path.join(examples_path, example_name)), cv2.COLOR_BGR2GRAY)
    scale_percent = 5
    # SE CALCULAN LAS NUEVAS DIMENSIONES DE LA IMAGEN
    width = int(example.shape[1] * scale_percent / 100)
    height = int(example.shape[0] * scale_percent / 100)
    dim = (width, height)
    # SE RESCALA LA IMAGEN
    resized_example = cv2.resize(example, dim, interpolation = cv2.INTER_AREA)
    kp, des = sift.detectAndCompute(resized_example, None)
    sift_points_examples_list.append(des)
pos2nb_dict = dict(zip(np.arange(len(folder_names_list)), folder_names_list))

In [23]:
img_paths = list()
for folder in os.listdir(data_path):
    for image in os.listdir(os.path.join(data_path, folder)):
        img_paths.append([os.path.join(data_path, folder, image), folder])
df = pd.DataFrame(img_paths, columns = ["image_path", "folder"])
del img_paths

In [24]:
sift_ponts = list()
for image_path in df.image_path:
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2GRAY)
    kp, des = sift.detectAndCompute(image, None)
    sift_ponts.append(des)
df["sift_points"] = sift_ponts
del sift_ponts

In [25]:
df.head()

,image_path,folder,sift_points
0,../Circuito_Almeria_171021/Iniciados/Data_resi...,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0..."
1,../Circuito_Almeria_171021/Iniciados/Data_resi...,1,"[[5.0, 1.0, 1.0, 1.0, 1.0, 57.0, 77.0, 17.0, 3..."
2,../Circuito_Almeria_171021/Iniciados/Data_resi...,1,"[[17.0, 62.0, 166.0, 153.0, 2.0, 2.0, 2.0, 4.0..."
3,../Circuito_Almeria_171021/Iniciados/Data_resi...,1,"[[0.0, 0.0, 0.0, 0.0, 10.0, 64.0, 17.0, 0.0, 1..."
4,../Circuito_Almeria_171021/Iniciados/Data_resi...,1,"[[1.0, 24.0, 23.0, 7.0, 9.0, 13.0, 7.0, 2.0, 6..."


In [26]:
def sift_cluster_one_example(sift_points_data, sift_points_examples_list = sift_points_examples_list):
    nb_good_matches = list()
    for sift_points_example in sift_points_examples_list:
        # BFMatcher(Brute Force Matcher) with defalut setting
        bf = cv2.BFMatcher(cv2.NORM_L2)
        matches = bf.knnMatch(sift_points_data, sift_points_example, k=2)
        # Apply ratio test as in David Rowe's paper
        good_matches = []
        for m, n in matches:
            if m.distance < 0.75 * n.distance:
                good_matches.append(m)
        nb_good_matches.append(len(good_matches))
    return(np.argmax(nb_good_matches))

In [27]:
sift_cluster_one_example(df.sift_points[2002])

23

In [28]:
pred_folder = list()
for sift_points in df.sift_points:
    pred_folder.append(pos2nb_dict[sift_cluster_one_example(sift_points)])

In [29]:
df["pred_folder"] = pred_folder
accuracy = (sum(df.folder == df.pred_folder)-34)/(len(df)-34)
print("Accuracy: " + str(round(accuracy*100, 2)) + "%")

Accuracy: 10.4%


In [30]:
from sklearn.metrics import confusion_matrix
confusion_matrix(df.folder, df.pred_folder)

array([[ 4, 14,  0, ...,  1,  1,  1],
       [ 3, 17,  0, ...,  1,  0,  0],
       [ 2, 20, 15, ...,  0,  0,  0],
       ...,
       [ 1,  7,  0, ..., 15,  0,  0],
       [ 2, 15,  0, ...,  0,  3,  0],
       [ 1,  9,  0, ...,  4,  2,  3]], dtype=int64)

In [13]:
np.random.seed(42)
rnd_folder = np.random.randint(1,35, len(df)).astype(str)
df["rnd_folder"] = pd.Series(rnd_folder)
accuracy_rnd = sum(df.folder == df["rnd_folder"])/len(df)
print("El modelo lo hace " + str(round(accuracy/accuracy_rnd,2)) + " veces mejor que la asignación aleatoria")

El modelo lo hace 4.07 veces mejor que la asignación aleatoria


In [14]:
examples2_path = "../Circuito_Almeria_171021/Iniciados/Examples2/"

In [15]:
sift_points_examples_list = list()
folder_names_list = list()
for folder in os.listdir(examples2_path):
    folder_names_list.append(folder)
    for image in os.listdir(os.path.join(examples2_path, folder)):
        example = cv2.imread(os.path.join(examples2_path, folder, image))
        scale_percent = 5
        # SE CALCULAN LAS NUEVAS DIMENSIONES DE LA IMAGEN
        width = int(example.shape[1] * scale_percent / 100)
        height = int(example.shape[0] * scale_percent / 100)
        dim = (width, height)
        # SE RESCALA LA IMAGEN
        resized_example = cv2.resize(example, dim, interpolation = cv2.INTER_AREA)
        kp, des = sift.detectAndCompute(resized_example, None)
        sift_points_examples_list.append(des)

In [16]:
def sift_cluster_two_examples_top(sift_points_data, sift_points_examples_list = sift_points_examples_list):
    nb_good_matches = list()
    for sift_points_example in sift_points_examples_list:
        # BFMatcher(Brute Force Matcher) with defalut setting
        bf = cv2.BFMatcher(cv2.NORM_L2)
        matches = bf.knnMatch(sift_points_data, sift_points_example, k=2)
        # Apply ratio test as in David Rowe's paper
        good_matches = []
        for m, n in matches:
            if m.distance < 0.75 * n.distance:
                good_matches.append(m)
        nb_good_matches.append(len(good_matches))
    return(np.argmax(nb_good_matches))

In [17]:
sift_cluster_two_examples_top(df.sift_points[123])

57

In [18]:
pred_folder = list()
for sift_points in df.sift_points:
    pred_folder.append(pos2nb_dict[int(sift_cluster_two_examples_top(sift_points)/2)])

In [19]:
df["pred_folder_2"] = pred_folder
accuracy_2 = (sum(df.folder == df.pred_folder_2)-34*2)/(len(df)-34*2)
print("Accuracy: " + str(round(accuracy_2*100, 2)) + "%")

Accuracy: 11.47%


In [20]:
print("El modelo lo hace " + str(round(accuracy_2/accuracy_rnd,2)) + " veces mejor que la asignación aleatoria")

El modelo lo hace 4.66 veces mejor que la asignación aleatoria
